In [1]:
##you must change epoch and loss_threshold. You can see it in the relevant place.
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

url = 'https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/6_gradient_descent/insurance_data.csv'

In [2]:
df = pd.read_csv(url)
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']], df.bought_insurance, test_size = 0.2, random_state =25)

In [5]:
len(X_train)

22

In [6]:
df.shape

(28, 3)

In [14]:
#Scale
#why we do scale ?
#because bring the age and affordibility on a same scale
#if we do scale, machine learning model tends to work better
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [13]:

X_train_scaled.head()

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1


In [68]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,),activation='sigmoid',
                      kernel_initializer='ones',bias_initializer ='zeros')
])

model.compile(optimizer ='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

#epoch must be 5000
model.fit(X_train_scaled,y_train, epochs=5)

Epoch 1/5
1/1 [==============================] - 0s 344ms/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 2ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 1ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/5
1/1 [==============================] - 0s 2ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/5
1/1 [==============================] - 0s 2ms/step - loss: 0.7098 - accuracy: 0.5000


In [21]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 139ms/step - loss: 0.3550 - accuracy: 1.0000


[0.35497748851776123, 1.0]

# Lets check from x_test_scaled


In [28]:
#1-look x_test_scaled
X_test_scaled


,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [29]:
#2-make predict
model.predict(X_test_scaled)

array([[0.70548487],
       [0.3556955 ],
       [0.16827849],
       [0.47801173],
       [0.7260697 ],
       [0.8294984 ]], dtype=float32)

In [30]:
#3-look y_test(true)
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

# Look final weight(w) and bias(b)

In [33]:
#Tensorflow method
#weights = coef
#bias = intercept

coef, intercept = model.get_weights()
coef,intercept

(array([[5.060867 ],
        [1.4086502]], dtype=float32),
 array([-2.9137027], dtype=float32))

In [40]:
def sigmoid(x):
    import math
    return 1 / (1 + math.exp(-x))

# Instead of model.predict, write our own prediction function that uses w1,w2 and bias

In [41]:
# we can use w1,w2 and bias
def prediction_function(age,affordibility):
    weighted_sum = coef[0]* age + coef[1] * affordibility + intercept
    return sigmoid(weighted_sum)

In [42]:
#Lets make prediction, we use X_test_scaled[0]
#you can look upside
prediction_function(0.47,1)

0.7054848693136117

# Building a neural network from strach without tensorflow

In [43]:
#Cost Function log_loss
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [44]:
#sigmoid numpy is a vector type of function
#instead of taking 1 value this function takes an array
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [64]:
#
#Define Gradient Descent
#

def gradient_descent(age,affordability, y_true, epochs,loss_thresold):
    #w1,w2,bias
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        
        loss = log_loss(y_true, y_predicted)
        
        w1d = (1/n) * np.dot(np.transpose(age), (y_predicted - y_true))
        w2d = (1/n) * np.dot(np.transpose(affordability),(y_predicted-y_true)) 
        bias_d = np.mean(y_predicted-y_true)
        
        w1 = w1 - (rate * w1d)
        w2 = w2 - (rate * w2d)
        bias = bias - (rate * bias_d)
        
        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
        
        if loss<=loss_thresold:
            break

    return w1, w2, bias
        

In [69]:
#loss_threshold must be 0.4631
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train , 1000,loss_thresold=0.681264778737757)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss:0.681264778737757


(0.9556229728273669, 0.9058873696677865, -0.2122349122718517)

In [67]:
#Lets compare with tensorflow
coef, intercept

(array([[5.060867 ],
        [1.4086502]], dtype=float32),
 array([-2.9137027], dtype=float32))